In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import hvplot.pandas

ModuleNotFoundError: No module named 'hvplot'

In [ ]:
from os import listdir
from os.path import isfile

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
dir = "../Resources/"

In [ ]:
all_files = [x for x in listdir(dir) if (".xlsx" in x or ".csv" in x)]
all_files

In [ ]:
# pip install xlrd==1.2.0

In [ ]:
all_df = pd.DataFrame()
for x in all_files:
    a = x.replace(".xlsx", "")
    df = pd.read_excel(dir+x)
    df["name"] = a
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.set_index("Date")
    df = df.sort_index()
    df = df.dropna()
    df = df[df['Volume'] != 0.0]
    df.drop(['Volume'], axis=1, inplace=True)
    if a == "corn":
        corn = df
        all_df["corn"] = corn["Price"]
    elif a == "live_cattle":
        live_cattle = df
        all_df["live_cattle"] = live_cattle["Price"]
    elif a == "feeder_cattle":
        feeder_cattle = df
        all_df["feeder_cattle"] = feeder_cattle["Price"]
    elif a == "soybean_meal":
        soybean = df
        all_df["soybean"] = soybean["Price"]
    elif a == "lean_hogs":
        hog = df
        all_df["hog"] = hog["Price"]

In [ ]:
all_df = all_df.dropna()
corrM = all_df.corr()

mask = np.zeros_like(corrM)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(10,7))
    ax = sns.heatmap(corrM, mask=mask, center=0, annot=True, square=True, linewidths=1, cmap="seismic")

In [ ]:
# all_df["corn_and_soybean"] = ((all_df["corn"]*10) + (all_df["soybean"]*0.75))
all_df['hog_delta_5'] = all_df["hog"].shift(105)
all_df['corn_delta_5'] = all_df["corn"].shift(105)
all_df['soybean_delta_5'] = all_df["soybean"].shift(105)

In [ ]:
# all_df2 = all_df.pct_change(7)
all_df2 = all_df.dropna()

In [ ]:
# Crush Margin = 2 x CME lean hog futures price (5 months
# from placement) – 0.5 x CME lean hog futures price (5
# months from placement) – 10 x CME corn futures price (at
# placement) – 0.075 x CME soybean meal futures price (at
# placement)
# GFM t= 2.05 * LH t - WP t-5-(10 * C t-5)-(0.075 * SM t-5)
all_df["GFM"] = (2.05*all_df["hog"]) - (all_df['hog_delta_5']*0.5) - (10 * all_df['corn_delta_5']) - (0.075 * all_df['soybean_delta_5'])
# all_df2["GFM"] = (2.05*all_df2["hog"]) - (10 * all_df2['corn_delta_5']) - (0.075 * all_df2['soybean_delta_5'])
all_df2 = all_df2.drop(columns=['feeder_cattle', 'live_cattle', 'hog_delta_5', 'corn_delta_5', 'soybean_delta_5'])
all_df2.head(10)

In [ ]:
all_df2.plot(figsize=(15,10));

In [ ]:
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
#     for i in range(len(df) - window - 1):
    for i in range(window, len(df)):
#         features = df.iloc[i:(i + window), feature_col_number]
#         target = df.iloc[(i + window), target_col_number]
        features = df.iloc[i-window:i, feature_col_number]
        target = df.iloc[i, target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [ ]:
window_size = 10

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 3
target_column = 2
X, y = window_data(all_df2, window_size, feature_column, target_column)

In [ ]:
y

In [ ]:
pd.DataFrame(X)

In [ ]:
all_df2.head()

In [ ]:
all_df2.pct_change(2)

In [ ]:
split = int(0.7 * len(X))
X_train = X[: split - 1]
X_test = X[split:]
y_train = y[: split - 1]
y_test = y[split:]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Use the MinMaxScaler to scale data between 0 and 1.
scaler = MinMaxScaler()

scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [ ]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
model = Sequential()

number_units = 30
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units = number_units,
    return_sequences = True,
    input_shape = (X_train.shape[1],1))
         )
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(LSTM(
    units = number_units,
    return_sequences = True,
        ))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(
    units = number_units,
    return_sequences = False,
        ))
model.add(Dropout(dropout_fraction))

model.add(Dense(1))

In [ ]:
model.compile(optimizer="adam", loss="mean_squared_error")
# model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
epochs = 10
batch_size = 10
model.fit(X_train, y_train, epochs=epochs, shuffle=False, batch_size=batch_size, verbose=1)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_test

In [ ]:
predicted = model.predict(X_test)

In [ ]:
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
hogs = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices): ]) 
hogs.head()

In [ ]:
hogs.hvplot.line(xlabel="Date", ylabel="Price")